### 1. Librerias 

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

### 2. Función para extraer los datos(Web scarping)

In [2]:
def extract_text(id, url, by_type, identifier):
    # Configurar el servicio para ChromeDriver
    service = Service(ChromeDriverManager().install())

    # Iniciar el navegador
    driver = webdriver.Chrome(service=service)

    try:
        # Navegar a la página web
        print(f"Navegando a la URL: {url}")
        driver.get(url)
        
        time.sleep(2)# pausa de 2 segundos
        # Hacer scroll hacia abajo para cargar el contenido si es necesario
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Esperar hasta que el contenedor de los productos esté presente
        wait = WebDriverWait(driver, 5)  # Incrementar el tiempo de espera
        print(f"Esperando el elemento con {by_type} y {identifier}")
        wait.until(EC.presence_of_element_located((by_type, identifier)))

        # Extraer datos utilizando el tipo de localización y el identificador proporcionado
        element = driver.find_element(by_type, identifier)
        print(f"Elemento encontrado: {element}")

        # Obtener el texto del elemento
        text = element.text
        print(f"Texto encontrado para {id}: {text}")
        text_name = (id, text)
        
        return text_name
    except Exception as e:
        print(f"Error al extraer texto para {id} con URL {url}: {e}")
        return None
    finally:
        # Asegurarse de que el navegador se cierra
        driver.quit()





### 3. Creación de los argumentos que se van a pasar a la función

In [5]:
# id, empresa, producto, url, xpath
links_products = [
      ['1', 'Carrefour', 'azucar', 'https://www.carrefour.com.ar/azucar-comun-tipo-a-bella-vista-1-kg-421740/p', '/html/body/div[3]/div/div[1]/div/div/div/div[5]/div/div[2]/div[3]/section/div/div[2]/div/div[10]/div/div/div/div/div/div[2]/div/div/div[1]/div/div[1]/span/span/span[1]'],
      ['2', 'Dia', 'azucar', 'https://diaonline.supermercadosdia.com.ar/az%C3%BAcares?_q=az%C3%BAcares&map=ft', '/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[5]/section/a/article/div/div/div[5]/div/div/div[1]/span/span/span'],
      ['3', 'Disco', 'azucar', 'https://www.disco.com.ar/azucar?_q=azucar&map=ft&page=1', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[1]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div']
]

In [7]:
# Se crea un dataframe con la lista links_products
columnas = ['id','empresa', 'producto','link', 'xpath']
df = pd.DataFrame(links_products, columns=columnas)
df


,id,empresa,producto,link,xpath
0,1,Carrefour,azucar,https://www.carrefour.com.ar/azucar-comun-tipo...,/html/body/div[3]/div/div[1]/div/div/div/div[5...
1,2,Dia,azucar,https://diaonline.supermercadosdia.com.ar/az%C...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
2,3,Disco,azucar,https://www.disco.com.ar/azucar?_q=azucar&map=...,/html/body/div[2]/div/div[1]/div/div[10]/div/d...


### 4. Llamada a la funcion extract_text(id, url, by_type, identifier)

In [8]:
# Se extrae la data de la web
productos = []
# se recorre el dataframe df
for index, row in df.iterrows():
    # variables que se le pasan a la función
    id = row['id']
    url = row['link']
    xpath = row['xpath']
    
    result = extract_text(id, url, By.XPATH, xpath) 
    if result:  # Solo añadir si no es None
        productos.append(result)
    
    # Esperar 5 segundos antes de la siguiente iteración
    print("Esperando 1 segundos antes de la siguiente iteración...")
    # time.sleep(5)

# Imprimir el diccionario resultante
print(productos)
    

Navegando a la URL: https://www.carrefour.com.ar/azucar-comun-tipo-a-bella-vista-1-kg-421740/p
Esperando el elemento con xpath y /html/body/div[3]/div/div[1]/div/div/div/div[5]/div/div[2]/div[3]/section/div/div[2]/div/div[10]/div/div/div/div/div/div[2]/div/div/div[1]/div/div[1]/span/span/span[1]
Elemento encontrado: <selenium.webdriver.remote.webelement.WebElement (session="a15a55b632d6ba0bd374ae79c06453c5", element="f.4137261A13BFF631142AA8C0901FE478.d.DEE5AA5904579D6587098EB715B1C31F.e.173")>
Texto encontrado para 1: $ 860,00
Esperando 1 segundos antes de la siguiente iteración...
Navegando a la URL: https://diaonline.supermercadosdia.com.ar/az%C3%BAcares?_q=az%C3%BAcares&map=ft
Esperando el elemento con xpath y /html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[5]/section/a/article/div/div/div[5]/div/div/div[1]/span/span/span
Elemento encontrado: <selenium.webdriver.remote.webelement.WebElement (session="4abb81

In [9]:
# Crear el DataFrame
df_precio = pd.DataFrame(productos, columns=['id', 'precio'])
df_precio

,id,precio
0,1,"$ 860,00"
1,2,"$ 1.390,00"
2,3,$800
